In [24]:
#installing necessary packages
import sys
stdout = sys.stdout

sys.stdout = stdout
#!{sys.executable} -m pip install python-Levenshtein
!{sys.executable} -m pip install bert-serving-server bert-serving-client
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

You should consider upgrading via the 'C:\Users\joshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\joshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\joshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [52]:
import numpy as np
import pandas as pd
import re

data = pd.read_csv('test.csv')

# execute chosen function and print results
def getResults(questions, fn):
    def getResult(q):
        [answer, score, prediction] = fn(q)
        return [q, prediction, answer, score]

    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])

# test set of questions
test_data = [
    "What is the population of Egypt?",
    "What is the poplation of egypt",
    "How long is a leopard's tail?",
    "Do you know the length of a leopard's tail?",
    "biggest city in Finland"
]
    


In [53]:
def getNaiveAnswer(q):
    # regex helps to pass some punctuation signs
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    return "Sorry, I didn't get you.", 0, ""


In [54]:
getResults(test_data, getNaiveAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,78 million,1
1,What is the poplation of egypt,,"Sorry, I didn't get you.",0
2,How long is a leopard's tail?,How long is a leopard's tail?,60-110cm,1
3,Do you know the length of a leopard's tail?,,"Sorry, I didn't get you.",0
4,biggest city in Finland,What is the biggest city in Finland?,Greater Helsinki,1


In [55]:
def lratio(s, t):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # Else cost is 1; calculate ratio
                cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions

    ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
    return ratio

In [57]:
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = lratio(row["Question"], q)
        if score >= 0.9:  # I'm sure, stop here 
            return  row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]

    if max_score > 0.8:
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction

getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,78 million,1.000000
1,What is the poplation of egypt,What is the population of Egypt?,78 million,0.951613
2,How long is a leopard's tail?,How long is a leopard's tail?,60-110cm,1.000000
3,Do you know the length of a leopard's tail?,How long is a leopard's tail?,"Sorry, I didn't get you.",0.750000
4,biggest city in Finland,What is the biggest city in Finland?,"Sorry, I didn't get you.",0.779661


In [ ]:
def improvedApproxAnswer(q):
    # if question matches all or part of the original question, score = 1
    s = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(s) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]